# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [6]:
from sklearn.model_selection import train_test_split
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X_train shape: (22792, 14), Y_train shape: (22792,)
X_test shape: (9769, 14), Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

random_state provides control over the randomness in the data splitting process, promoting reproducibility, comparability, and ensuring the integrity of machine learning experiments and evaluations. 
It's a good practice to always specify random_state when splitting data to maintain consistency in results.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Define transformers for numerical and categorical features
num_transforms = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

cat_transforms = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, numeric_features),
        ('cat_transforms', cat_transforms, categorical_features)
    ])


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [11]:
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

pipeline.fit(X_train, Y_train)

Y_pred = pipeline.predict(X_test)

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [14]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss

scoring = {
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

cv_results = cross_validate(pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)


c:\ProgramData\Anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\ProgramData\Anaconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training results:
Negative Log Loss: 0.3674
ROC AUC: 0.9035
Accuracy: 0.8520
Balanced Accuracy: 0.7734


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [23]:

fold_results = pd.DataFrame( {
    'Train Negative Log Loss': -cv_results['train_neg_log_loss'],
    'Test Negative Log Loss': -cv_results['test_neg_log_loss'].mean(),
    'Train ROC AUC': -cv_results['train_neg_log_loss'],
    'Test ROC AUC': -cv_results['test_neg_log_loss'],
    'Train Accuracy': cv_results['train_roc_auc'],
    'Test Accuracy': cv_results['test_roc_auc'],
    'Train Balanced Accuracy': cv_results['train_accuracy'],
    'Test Balanced Accuracy': cv_results['test_accuracy']
})

fold_results_sorted = fold_results.sort_values(by='Test Negative Log Loss')

print(fold_results_sorted)

   Train Negative Log Loss  Test Negative Log Loss  Train ROC AUC  \
0                 0.081443                0.367379       0.081443   
1                 0.081137                0.367379       0.081137   
2                 0.081166                0.367379       0.081166   
3                 0.082115                0.367379       0.082115   
4                 0.081553                0.367379       0.081553   

   Test ROC AUC  Train Accuracy  Test Accuracy  Train Balanced Accuracy  \
0      0.337201             1.0       0.904550                 0.999945   
1      0.351998             1.0       0.903585                 1.000000   
2      0.367559             1.0       0.902240                 1.000000   
3      0.385017             1.0       0.906000                 1.000000   
4      0.395121             1.0       0.901040                 0.999945   

   Test Balanced Accuracy  
0                0.847554  
1                0.849309  
2                0.852348  
3                0.855

Calculate the mean of each metric. 

In [24]:
mean_metrics = {
    'Mean Train Negative Log Loss': -cv_results['train_neg_log_loss'].mean(),
    'Mean Test Negative Log Loss': -cv_results['test_neg_log_loss'].mean(),
    'Mean Train ROC AUC': cv_results['train_roc_auc'].mean(),
    'Mean Train Accuracy': cv_results['train_accuracy'].mean(),
    'Mean Train Balanced Accuracy': cv_results['train_balanced_accuracy'].mean(),
    'Mean Test ROC AUC': cv_results['test_roc_auc'].mean(),
    'Mean Test Accuracy': cv_results['test_accuracy'].mean(),
    'Mean Test Balanced Accuracy': cv_results['test_balanced_accuracy'].mean()
}

print(pd.Series(mean_metrics))

Mean Train Negative Log Loss    0.081483
Mean Test Negative Log Loss     0.367379
Mean Train ROC AUC              1.000000
Mean Train Accuracy             0.999978
Mean Train Balanced Accuracy    0.999955
Mean Test ROC AUC               0.903483
Mean Test Accuracy              0.851966
Mean Test Balanced Accuracy     0.773374
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [19]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

Y_pred_proba = pipeline.predict_proba(X_test)
Y_pred = pipeline.predict(X_test)

metrics = {
    'Negative Log Loss': log_loss(Y_test, Y_pred_proba),
    'ROC AUC': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'Accuracy': accuracy_score(Y_test, Y_pred),
    'Balanced Accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

print(metrics)

{'Negative Log Loss': 0.36870848123544403, 'ROC AUC': 0.9019831463572563, 'Accuracy': 0.8547446002661481, 'Balanced Accuracy': 0.7748948893592034}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

immediately recoding the target variable income right after loading the data simplifies data handling, improves model compatibility, and enhances interpretability of results, making the entire data analysis pipeline more efficient and reliable.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.